# 0. Initialization
To get started we need to import relevant libraries and a valid OpenAI key.
For convenience we also define a helper function to simplify calling the OpenAI ChatCompletion API.

In [1]:
# OpenAI initialisation and additional dependencies
import os
import openai
from pprint import pprint
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
# Helper function for calling ChatCompletion API

# Alternative models to try: gpt-4-1106-preview, gpt-4, gpt-3.5-turbo-0125, gpt-3.5-turbo, gpt-3.5-turbo-0613

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# 1. Three key prompt engineering strategies

## 1.1 Zero-Shot Prompting

Large LLMs today, such as ChatGPT, are tuned to follow instructions and are trained on large amounts of data. Therefore, they are capable of performing some tasks "zero-shot.". This means that the model can perform a task without any fine-tuning or training on the task. 

In [3]:
# Zero-shot example: translation

prompt = f"""
Translate the following English text to Spanish and English pirate: \ 
```Hello, I would like to purchase an insurance package.```
"""

Note that in the prompt above we didn't provide the model with any examples, the LLM already understands the relevant aspects. That's the zero-shot capabilities at work.


When zero-shot doesn't work, it's recommended to provide demonstrations or examples in the prompt which leads us to few-shot prompting.

## 1.2 Few-Shot Prompting

While LLMs demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using zero-shot prompting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

Lets say you are a basketball manufacturer and want to make a sentiment analysis based on customer reviews. You can use few-shot prompting to steer the model to better performance. Here is an example of how you can do that:

In [4]:
# Zero-shot attempt
prompt = f"""
Determine the sentiment of this sentence.
Sentence: This basketball has a lot of weight.
"""

response = get_completion(prompt)
print(response)

Neutral


Even though the model classifies the review as neutral, a basketball with a lot of weight might be a faulty product. Therefore, we can use few-shot prompting to steer the model to better performance. 

In [5]:
# Few-shot attempt
prompt = f"""
Sentence: This basketball is easy to carry.

Answer: The sentiment of this sentence is positive.

Determine the sentiment of the sentence.

Sentence: This basketball has a lot of weight.

Answer:
"""

response = get_completion(prompt)
print(response)

The sentiment of this sentence is negative.


This time the model classifies the review as 'negative' as it uses the provided example as a reference.

Even though few-shot prompting is a powerful technique, it has some limitations - as shown here:

In [6]:
# Unsuccessful few-shot attempt at average calculation
prompt = f"""
You are an AI assistant capable of performing complex  numerical computations.

Question 1:
Find the average of the numbers. Here are the numbers: 1, 2, 3.

Answer 1:
The average is 2.

Question 2:
Find the average of the numbers. Here are the numbers: 5, 10, 15, 7.

Answer 2:
The average is 9.25.

Question 3:
Find the average of the numbers. Here are the numbers: 63409, 95328, 45860, 91378, 210691, 50923, 50760, 99145.
"""

response = get_completion(prompt)
print(response)


Answer 3:
The average is 77906.5.


In [7]:
# checking the calculation: 
x = [63409, 95328, 45860, 91378, 210691, 50923, 50760, 99145]
sum(x)/len(x)

88436.75

As we can see this is not the correct response and there is a need for more advanced prompt engineering.

## 1.3 Chain-of-thoughts prompting

Chain of thought prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In [8]:
# Average calculation with combined few-shot and chain-of-thoughts prompting
prompt = f"""
You are an AI assistant capable of performing complex  numerical computations.

Question 1:
Find the average of the numbers. Here are the numbers: 1, 2, 3.

Answer 1:
We start by finding the sum of the numbers. 1+2=3, 3+3=6. The sum of the numbers is 6. Then we divide the sum by the number of numbers. 6/3=2. The average is 2.

Question 2:
Find the average of the numbers. Here are the numbers: 5, 10, 15, 7.

Answer 2:
We start by finding the sum of the numbers. 5+10=15, 15+15=30, 30+7=37. The sum of the numbers is 37. Then we divide the sum by the number of numbers. 37/4=9.25. The average is 9.25.

Question 3:
Find the average of the numbers. Here are the numbers: 63409, 95328, 45860, 91378, 210691, 50923, 50760, 99145.
"""

response = get_completion(prompt)
pprint(response)

('Answer 3:\n'
 'We start by finding the sum of the numbers. \n'
 '63409+95328=158737\n'
 '158737+45860=204597\n'
 '204597+91378=295975\n'
 '295975+210691=506666\n'
 '506666+50923=557589\n'
 '557589+50760=608349\n'
 '608349+99145=707494\n'
 '\n'
 'The sum of the numbers is 707494. \n'
 'Then we divide the sum by the number of numbers. \n'
 '707494/8=88436.75\n'
 'The average is 88436.75.')


As we can see, the model is able to reason about the question and provide a correct answer when given more detailed instructions on how to solve the problem.

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# 2. Basic prompt engineering principles, with some illustrations 

## 2.1 Principle 1: Write clear and specific instructions

### Tactic 1 - Use delimiters to clearly indicate distinct parts of the input
By using delimiters you can clearly indicate distinct parts of the input. Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`.

In [9]:
# Delimiter usage example

text = """
For a model to work well, just be really clear \ 
about what you want. Give specific instructions \ 
so the model knows exactly what to do. This helps \ 
get the right answers and avoids mistakes. Remember, \ 
a short prompt isn't always a clear one. Sometimes, \ 
it's better to use longer prompts to give more \ 
info to the model. This usually leads to better \ 
and more relevant results.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

response = get_completion(prompt)
print(response)

To ensure a model works effectively, provide clear and specific instructions to avoid mistakes and achieve accurate results, even if it means using longer prompts for better outcomes.


### Tactic 2 - Ask for a structured output
By asking for a structured output you can get output that is easier to parse and use in your application. This is especially useful when you want to use the output in a downstream task. Examples of structured output are JSON and HTML.

In [10]:
# Example asking for structured (JSON) output
prompt = f"""
Generate a list of three fictional banking product names along with their developers and categories. 
Provide them in JSON format with the following keys: product_id, name, developer, category.
"""

response = get_completion(prompt)
print(response)

[
    {
        "product_id": 1,
        "name": "CashFlow Pro",
        "developer": "FinTech Innovations Inc.",
        "category": "Personal Finance Management"
    },
    {
        "product_id": 2,
        "name": "SecureSave",
        "developer": "CyberGuard Solutions",
        "category": "Online Banking Security"
    },
    {
        "product_id": 3,
        "name": "InvestWise",
        "developer": "WealthTech Solutions",
        "category": "Investment Management"
    }
]


### Tactic 3 - Ask the model to check whether conditions are satisfied
The model can be asked to make reasoning steps, for example check if a condition is satisfied and perform an action depending on the result. 

In [11]:
# Example asking the model to check a condition - part 1 - condition met
text_1 = f"""
Understanding Insurance Basics is Simple! First, identify the type of coverage you need. 
While doing that, gather relevant information about your assets and risks. 
Next, choose a reputable insurance provider and select a suitable policy. 
Once you've made your decision, go ahead and complete the application process. 
Wait for the policy to be approved. After approval, review the terms and conditions carefully. 
If everything looks good, make the required payments to activate your coverage. 
If needed, consult with an insurance agent for clarification. 
And there you have it! You've successfully secured insurance coverage for your peace of mind.
"""

# prompt with a conditional instruction + text input
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)
print(response)


Step 1 - Identify the type of coverage you need.
Step 2 - Gather relevant information about your assets and risks.
Step 3 - Choose a reputable insurance provider and select a suitable policy.
Step 4 - Complete the application process.
Step 5 - Wait for the policy to be approved.
Step 6 - Review the terms and conditions carefully after approval.
Step 7 - Make the required payments to activate your coverage.
Step 8 - Consult with an insurance agent for clarification if needed.


In [12]:
# Example asking the model to check a condition - part 2 - condition not met
text_2 = f"""
The stock market is buzzing with activity today, and financial analysts are deep in discussions. 
It's an intriguing day to explore the dynamics of the financial world. 
The charts are fluctuating, and economic indicators are being closely monitored. 
Traders are making strategic moves, and investors are assessing their portfolios. 
Some are engaged in intense conversations about market trends, while others are researching potential investment opportunities. 
It's an ideal day to delve into the intricacies of finance and witness the ebb and flow of economic activities.
"""

# prompt with a conditional instruction + text input
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt)
print(response)

No steps provided.


### Tactic 4 - Providing examples
By giving the model a few examples of the desired output, it can learn to generalize to similar cases.

In [13]:
# Providing a sample conversation
prompt = f"""
Your task is to answer in a consistent style.

<client>: Enlighten me about financial planning.

<financial_advisor>: The fortune that grows most steadily \
emanates from disciplined savings; the \
most prosperous investment portfolio begins with a single asset; \
the most secure financial future starts with a well-laid plan.

<client>: Enlighten me about risk management.
"""

response = get_completion(prompt)
print(response)

<financial_advisor>: Risk management is the process of identifying, assessing, and prioritizing risks followed by coordinated and economical application of resources to minimize, monitor, and control the probability and impact of unfortunate events. It is essential in ensuring the protection and sustainability of your financial assets.


## 2.2 Principle 2: Give the model time to “think” 

### Tactic 1 - Specify the steps required to complete a task
By specifying the steps required to complete a task, the model can learn to perform the task step-by-step instead of rushing to a conclusion.

In [14]:
# Example using complex (and slightly pointless) step by step instructions

text = f"""
In a bustling city, siblings Alex and Emma embarked on \
a mission to secure insurance coverage for their \
family. As they navigated the policy options, discussing earnestly, uncertainty \
struck—Alex faced unexpected challenges understanding the terms \
of coverage, and Emma encountered confusing jargon. \
Though slightly perplexed, the duo sought assistance from \
knowledgeable advisors. Despite the confusion, \
their determined efforts prevailed, and they \
left the insurance agency with a sense of security and understanding.
"""

# step-by-step instructions:
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print(response)

1 - Alex and Emma sought insurance coverage for their family, faced challenges understanding the terms, sought help from advisors, and left with a sense of security.

2 - Alex et Emma ont cherché une couverture d'assurance pour leur famille, ont rencontré des défis pour comprendre les termes, ont demandé de l'aide à des conseillers, et sont partis avec un sentiment de sécurité.

3 - Alex, Emma

4 - 
{
  "french_summary": "Alex et Emma ont cherché une couverture d'assurance pour leur famille, ont rencontré des défis pour comprendre les termes, ont demandé de l'aide à des conseillers, et sont partis avec un sentiment de sécurité.",
  "num_names": 2
}


### Tactic 2 - Instruct the model to work out its own solution before rushing to a conclusion


<center><img src="resources/Thinking-Fast-and-Slow.jpg" alt="Prompting principle 2: time to think"/>

We can all be prone to jumping to conclusions. The same is true for models. By instructing the model to work out its own solution before rushing to a conclusion, we can avoid this problem. In this example we can see the model being "tempted" by an easy solution...

In [15]:
# jumping to wrong conclusion
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.

Question:
```
I'm starting a financial consultancy and need assistance
working out the budget.
- Office space rental costs $50 / square foot 
- I can purchase computer equipment for $200 / square foot
- I negotiated an annual contract for IT support that will cost
me a flat $80k per year, and an additional $15 / square
foot
What is the total budget for the first year of operations
as a function of the number of square feet?
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Office cost: 50x
2. Solar panel cost: 200x
3. Maintenance cost: 80,000 + 150x
Total cost: 50x + 200x + 80,000 + 150x = 400x + 80,000

"""
response = get_completion(prompt)
pprint(response)

("The student's solution is incorrect. \n"
 '\n'
 'The correct total budget for the first year of operations as a function of '
 'the number of square feet should be:\n'
 'Total cost = Office cost + Computer equipment cost + IT support cost\n'
 'Total cost = $50x + $200x + $80,000 + $15x\n'
 'Total cost = $265x + $80,000\n'
 '\n'
 'Therefore, the correct total budget for the first year of operations as a '
 'function of the number of square feet is $265x + $80,000.')



Note that the student's solution is actually not correct. We can fix this by instructing the model to work out its own solution first.

In [16]:
# thinking things through, and getting it right
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm starting a financial consultancy and need assistance
working out the budget.
- Office space rental costs $50 / square foot 
- I can purchase computer equipment for $200 / square foot
- I negotiated an annual contract for IT support that will cost
me a flat $80k per year, and an additional $15 / square
foot
What is the total budget for the first year of operations
as a function of the number of square feet?
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Office cost: 50x
2. Solar panel cost: 200x
3. Maintenance cost: 80,000 + 150x
Total cost: 50x + 200x + 80,000 + 150x = 400x + 80,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let's calculate the total budget for the first year of operations as a function of the number of square feet.

Given:
- Office space rental costs $50 / square foot
- Computer equipment costs $200 / square foot
- IT support contract: $80,000 flat fee + $15 / square foot

Let x be the size of the installation in square feet.

Costs:
1. Office cost: $50x
2. Computer equipment cost: $200x
3. IT support cost: $80,000 + $15x

Total cost: $50x + $200x + $80,000 + $15x = $265x + $80,000

Therefore, the total budget for the first year of operations as a function of the number of square feet is $265x + $80,000.

Is the student's solution the same as the actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


# 3. Prompting examples for common LLM tasks

## 3.1 Summarizing

### Basic summarization using word, sentence or character limit

In [17]:
# Sample text for summary examples
prod_review = """
Acquired this 'FjordGuard Assurance' policy for my family's financial security, and they find it reassuring. 
The coverage is comprehensive, and the policy terms are transparent. The customer service has a welcoming approach, providing a sense of trust. 
However, I feel the premium is a bit high for the coverage offered. 
There might be other insurance options with more extensive benefits at a similar cost. 
Surprisingly, the policy documentation arrived a day earlier than expected, allowing me to review it thoroughly before presenting it to my family.
"""

In [18]:
# Straightforward summary
prompt = f"""
Your task is to generate a short summary of a product \
review. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
pprint(response, width=80)

("The 'FjordGuard Assurance' policy offers comprehensive coverage with "
 'transparent terms and excellent customer service, but the premium may be '
 'high compared to other options. Arrived early for review.')


### Summarize with a specific focus

We can ask for a summary with specific focus, getting a different response.

In [19]:
# Summary with specific focus
 
prompt = f"""
Your task is to generate a short summary of a product \
review to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
pprint(response, width=80)

('The policy documentation arrived earlier than expected, allowing for '
 'thorough review. Premium cost is perceived as high compared to coverage.')


As we can see the summaries include topics that are not related to the topic of focus. Changing the wording of the prompt can sometimes result in better results.

In [20]:
# Asking for information extract instead of a summary
prompt = f"""
Your task is to extract relevant information from \ 
a product review to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
pprint(response, width=80)

('Feedback: The customer received the policy documentation a day earlier than '
 'expected, which was appreciated. Consider exploring insurance options with '
 'more extensive benefits at a similar cost.')


## 3.2 Inferring and categorising

A standard NLP problem is sentiment analysis and we can use LLMs to perform this task. 

In [21]:
# Sample review text
review = """
Needed a reliable financial advisor for my investments, and NordWealth stood out with additional services and reasonable fees. After purchasing their comprehensive financial planning package, the onboarding process was swift. The communication channel broke during the initial consultation, and the company promptly arranged for a follow-up. The financial plan was delivered within a few days, and the advisor explained it thoroughly. Setting up the investment portfolio was a breeze. I encountered a minor issue, so I contacted their support, and they promptly resolved the matter. NordWealth appears to be an excellent financial service provider that prioritizes its clients and their financial goals!
"""

In [22]:
# Review sentiment determined using LLM
prompt = f"""
What is the sentiment of the following review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Positive


Another powerful application of LLMs is to extract information from reviews. This can be useful for example when you want to extract information about a product or service from reviews.

In [ ]:
# This prompt is taken directly from the DeepLearning course.
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{review}'''
"""

response = get_completion(prompt)
print(response)

LLMs are also useful for categorising and labelling text according to content, as illustrated below.

In [ ]:
# Sample input text
story = """
In a recent governmental survey examining employee satisfaction within the Norwegian financial landscape, participants were tasked with evaluating their contentment levels within their respective banking institutions. The findings illuminated that NorthHarbor Financial Alliance emerged as the leading institution, securing a commendable satisfaction rating of 85%.

One employee from NorthHarbor Financial Alliance, Isabella Nordstrøm, shared her perspective on the results, stating, "The recognition of NorthHarbor doesn't come as a surprise. It's a remarkable workplace with a collaborative culture and ample growth opportunities. I take pride in contributing to such a distinguished financial institution."

The positive outcomes were warmly embraced by NorthHarbor Financial Alliance's leadership, with CEO Anders Olsen expressing, "We are elated to witness such robust levels of satisfaction among our dedicated team. Our collective efforts consistently aim for excellence, and it's genuinely fulfilling to see our workforce's contributions acknowledged."

Conversely, the survey spotlighted that Nordic Finance Collective exhibited the lowest satisfaction rating, with only 38% of employees expressing contentment in their roles. In response, the government has committed to addressing the concerns raised in the survey and is actively working toward enhancing overall job satisfaction throughout the financial sector.
"""

In [ ]:
# Extracting topics from text using LLM
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""

response = get_completion(prompt)
print(response)

## 3.3 Transforming

LLM's are trained on many different sources in many languages - this gives the model the ability to do translations.

In [ ]:
# effortless translation
prompt = f"""
Translate the following English text to Spanish and English pirate: \ 
```Hello, I would like to purchase an insurance package.```
"""

response = get_completion(prompt)
pprint(response)

Writing can vary in tone and should be based on the intended audience. LLMs are very efficient in changing the tone of a given text.

In [ ]:
# changing tone of voice
prompt = f"""
Translate the following from slang to a business letter: 
'Dude! Joey here. Check out the specs for this savings account.'
"""

response = get_completion(prompt)
pprint(response)